In [2]:
import polars as pl

In [12]:
Pitching = pl.read_csv('./Data/lahman_1871-2023_csv/Pitching.csv')

In [14]:
Pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [15]:
Batting = pl.read_csv('./Data/lahman_1871-2023_csv/Batting.csv')
Batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [38]:
League_Wide_ERA = (Pitching
                     .select([pl.col('IPouts'), pl.col('yearID'), pl.col('R'), pl.col('lgID')])
                     .filter((pl.col('yearID') >= 1946))
                     .group_by(pl.col('yearID'), pl.col('lgID'))
                     .agg([pl.col('R').sum().alias('total_runs'),
                           pl.col('IPouts').sum().alias('total_ipouts')
                          ])
                     .with_columns(League_ERA = pl.col('total_runs')/pl.col('total_ipouts')*27)
                    )
League_Wide_ERA

yearID,lgID,total_runs,total_ipouts,League_ERA
i64,str,i64,i64,f64
1955,"""AL""",5491,33080,4.481771
1998,"""AL""",11354,60584,5.060049
2009,"""AL""",10793,60520,4.815119
1991,"""NL""",7955,52163,4.117574
2018,"""AL""",11075,64982,4.601659
…,…,…,…,…
2007,"""NL""",12388,69742,4.795905
2003,"""NL""",11917,69331,4.640911
1970,"""NL""",8771,52235,4.533684


In [116]:
League_Wide_ERA.head()

yearID,lgID,total_runs,total_ipouts,League_ERA
i64,str,i64,i64,f64
1955,"""AL""",5491,33080,4.481771
1998,"""AL""",11354,60584,5.060049
2009,"""AL""",10793,60520,4.815119
1991,"""NL""",7955,52163,4.117574
2018,"""AL""",11075,64982,4.601659


In [113]:
League_Wide_ERA.write_csv('League_Wide_ERA.csv')

In [81]:
DH = (Pitching
      .select([pl.col('IPouts'), pl.col('yearID'), pl.col('R'), pl.col('lgID')])
      .with_columns(
          (
              pl.when(pl.col('yearID') < 1972)
              .then(pl.lit('Neither'))
              .when(pl.col('yearID') < 2019)
              .then(pl.lit('Only AL'))
              .when(pl.col('yearID') == 2020)
              .then(pl.lit('Both'))
              .when(pl.col('yearID') == 2021)
              .then(pl.lit('Only AL'))
              .when(pl.col('yearID') >= 2022)
              .then(pl.lit('Both'))
              .otherwise(pl.lit(None))
              .alias('DH')
          )
      )
)

DH

IPouts,yearID,R,lgID,DH
i64,i64,i64,str,str
32,2004,8,"""NL""","""Only AL"""
159,2006,25,"""NL""","""Only AL"""
97,2007,24,"""AL""","""Only AL"""
146,2008,32,"""AL""","""Only AL"""
214,2009,23,"""AL""","""Only AL"""
…,…,…,…,…
207,1958,29,"""AL""","""Neither"""
39,1959,7,"""AL""","""Neither"""
55,2015,6,"""AL""","""Only AL"""


In [117]:
DH.head()

IPouts,yearID,R,lgID,DH
i64,i64,i64,str,str
32,2004,8,"""NL""","""Only AL"""
159,2006,25,"""NL""","""Only AL"""
97,2007,24,"""AL""","""Only AL"""
146,2008,32,"""AL""","""Only AL"""
214,2009,23,"""AL""","""Only AL"""


In [114]:
DH.write_csv('DH.csv')

In [82]:
Batting = pl.read_csv('./Data/lahman_1871-2023_csv/Batting.csv')

In [86]:
Batting.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [111]:
Slugging_Pct = (Batting
                     .select([pl.col('H'), pl.col('2B'), pl.col('3B'), pl.col('HR'), pl.col('AB'), pl.col('yearID')])
                     .filter((pl.col('yearID') >= 1946))
                     .group_by(pl.col('yearID'))
                     .agg([pl.col('2B').sum().alias('total_doubles'),
                           pl.col('3B').sum().alias('total_triples'),
                                 pl.col('HR').sum().alias('total_homeruns'),
                                       pl.col('H').sum().alias('total_Hits'),
                           pl.col('AB').sum().alias('total_AB')
                          ])
                .with_columns(total_singles = pl.col('total_Hits') - pl.col('total_doubles') - pl.col('total_triples')
                                                  - pl.col('total_homeruns'))
                     .with_columns(Slugging = (pl.col('total_singles')+(pl.col('total_doubles')*2)+(pl.col('total_triples')*3)+
                                   (pl.col('total_homeruns')*4))/(pl.col('total_AB'))
                    ))
Slugging_Pct

yearID,total_doubles,total_triples,total_homeruns,total_Hits,total_AB,total_singles,Slugging
i64,i64,i64,i64,i64,i64,i64,f64
1977,6441,1170,3644,38037,143975,26782,0.401111
1989,6307,868,3083,36293,142821,26035,0.37519
1998,8741,899,5064,44489,167116,29785,0.420187
2004,8919,898,5451,44522,167353,29254,0.427778
1971,4931,808,2863,32547,130544,23945,0.365264
…,…,…,…,…,…,…,…
1965,4199,787,2688,26952,109739,19278,0.371691
1947,3568,804,1565,22003,84436,16066,0.377493
1959,3478,591,2250,21636,84294,15317,0.392033


In [115]:
Slugging_Pct.write_csv('Slugging_Pct.csv')

In [118]:
Slugging_Pct.head()

yearID,total_doubles,total_triples,total_homeruns,total_Hits,total_AB,total_singles,Slugging
i64,i64,i64,i64,i64,i64,i64,f64
1977,6441,1170,3644,38037,143975,26782,0.401111
1989,6307,868,3083,36293,142821,26035,0.37519
1998,8741,899,5064,44489,167116,29785,0.420187
2004,8919,898,5451,44522,167353,29254,0.427778
1971,4931,808,2863,32547,130544,23945,0.365264
